# Обработка текста

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('SPAM text message 20170820 - Data.csv')

In [3]:
#df

In [4]:
df['Category'] = df['Category'].map({'ham':0, 'spam':1})

In [5]:
#df.Category.value_counts()

### Отложим часть текстов и меток на потом 

In [6]:
from sklearn.model_selection import train_test_split
X_1, X_long_box, y_1, y_long_box = train_test_split(df.Message, df.Category, test_size = 0.1, random_state = 0, stratify = df.Category)

In [7]:
# Осталосб для обучения и тестирования
#X_1.shape, y_1.shape 

## Подготовка текста


In [8]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\laptop\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [9]:
# фиксируем строку
letter = X_1.iloc[5]
letter

'Are you driving or training?'

In [10]:
# оставляем только латинские символы
letter = re.sub('[^a-zA-Z]', ' ', letter)
letter

'Are you driving or training '

In [11]:
# Нижний регистр
letter = letter.lower()
letter

'are you driving or training '

In [12]:
letter = letter.split()
letter

['are', 'you', 'driving', 'or', 'training']

In [14]:
# Создаем стеммер и выкидываем стоп слова
ps = PorterStemmer()
sw = stopwords.words('english')
#stopwords.remove('not')


In [15]:
#Удалим из нашего списка слов стоп слова и применим стеммер

letter1=[]
for word in letter:
    if word not in sw:
        stemmed = ps.stem(word)
        letter1.append(stemmed)

In [16]:
 letter1 

['drive', 'train']

In [17]:
' '.join(letter1)

'drive train'

In [18]:
def prepare_letter (letter):
    letter1 = []
    
    letter = re.sub('[^a-zA-Z]', ' ', letter)
    letter = letter.lower()
    letter = letter.split()
    for word in letter:
        if word not in sw:
            stemmed = ps.stem(word)
            letter1.append(stemmed)
    
    return ' '.join(letter1)
    

In [19]:
# Создадим словарь всех имеющихся писем в подготовленном виде
co = []
for i in range(0, X_1.shape[0]):
    co.append(prepare_letter(X_1.iloc[i]))

In [28]:
X_1.shape[0]

5014

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

# Векторизация нашего словаря с отбором 1500 наиболее часто встречающихся слов
cv = CountVectorizer(max_features = 1500)

X = cv.fit_transform(co).toarray()
y = y_1.values

In [21]:
X.shape

(5014, 1500)

## Данные подготовленны для использования в моделях машинного обучения

## Обучим классификатор

In [22]:
# Делим на тестовое и обучающее множество
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

In [23]:
X_train.shape

(4011, 1500)

In [29]:
# Не стесняясь берем случайный лес из коробки
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()

In [30]:
y_pred = clf.predict(X_test)


In [31]:
#Посмотрим на матрицу неточностей проверочного множества
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[867   2]
 [ 18 116]]


0.9800598205383848

In [196]:
# классификация единичного текста


In [32]:
text = X_long_box[1466]


In [33]:
text

'YOU 07801543489 are guaranteed the latests Nokia Phone, a 40GB iPod MP3 player or a £500 prize! Txt word:COLLECT to No:83355! TC-LLC NY-USA 150p/Mt msgrcvd18+'

In [34]:
# для прогноза новый текст нужно преобразовать в вектор, предварительно почистив о стоп-слов
v = prepare_letter(text)
#print (v)
vec = cv.transform([v])

In [35]:
clf.predict(vec)

array([1], dtype=int64)

In [36]:
for i in y_long_box.index:
    if y_long_box[i] == 1:
        print (i)

3639
5468
1469
833
962
1765
191
2626
1375
4410
3067
1137
2620
3807
2987
67
5004
3709
505
4725
4299
4901
3501
1993
5370
4042
4949
1466
3720
5012
5497
3463
4144
4213
4149
117
5112
3560
870
3423
5294
1699
648
3755
2071
3419
2113
591
5365
471
3483
4930
3571
3991
1544
4450
2094
717
3828
1072
529
2729
4198
4896
4473
3111
2670
4199
2770
4527
4967
850
4236
2767
592


## Сохранение модели

In [37]:
import pickle

pkl_filename = "forest_spam_model.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(clf, file)

In [38]:
pkl_filename = "vectorizer_spam_model.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(cv, file)

## Загрузка сохраненной модели

In [39]:
pkl_filename = 'vectorizer_spam_model.pkl'
with open(pkl_filename, 'rb') as file: 
    cv1 = pickle.load(file) 


In [40]:
pkl_filename = 'forest_spam_model.pkl'
with open(pkl_filename, 'rb') as file: 
    clf1 = pickle.load(file) 


In [42]:
# для прогноза новый текст нужно преобразовать в вектор, предварительно почистив о стоп-слов
v = prepare_letter(text)
print (v)
vec = cv1.transform([v])
clf1.predict(vec)

guarante latest nokia phone gb ipod mp player prize txt word collect tc llc ny usa p mt msgrcvd


array([1], dtype=int64)

## Пробуем класcифицировать нейросетью


In [43]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [44]:
X.max()

18

In [45]:
from sklearn.preprocessing import StandardScaler

In [46]:
SC = StandardScaler()
# внимание обучаем и трансформируем тренировочное множество
# а тестовое множество только трансформируем 
X_train = SC.fit_transform(X_train)
X_test = SC.transform(X_test)

In [47]:
model = models.Sequential()

model.add(layers.Dense(256, activation="relu", input_shape = (X_train.shape[1],)))
model.add(layers.Dense(1, activation="sigmoid",kernel_regularizer = regularizers.l2(0.001)))

In [48]:
# Компиляция модели
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [49]:
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=75, batch_size=32)

Epoch 1/75
126/126 [==============================] - 2s 8ms/step - loss: 0.5090 - accuracy: 0.8963 - val_loss: 0.1138 - val_accuracy: 0.9701
Epoch 2/75
126/126 [==============================] - 1s 7ms/step - loss: 0.0393 - accuracy: 0.9853 - val_loss: 0.0565 - val_accuracy: 0.9831
Epoch 3/75
126/126 [==============================] - 1s 5ms/step - loss: 0.0154 - accuracy: 0.9953 - val_loss: 0.0559 - val_accuracy: 0.9850
Epoch 4/75
126/126 [==============================] - 1s 5ms/step - loss: 0.0113 - accuracy: 0.9973 - val_loss: 0.0557 - val_accuracy: 0.9860
Epoch 5/75
126/126 [==============================] - 1s 6ms/step - loss: 0.0094 - accuracy: 0.9980 - val_loss: 0.0562 - val_accuracy: 0.9870
Epoch 6/75
126/126 [==============================] - 1s 5ms/step - loss: 0.0080 - accuracy: 0.9983 - val_loss: 0.0561 - val_accuracy: 0.9870
Epoch 7/75
126/126 [==============================] - 1s 5ms/step - loss: 0.0070 - accuracy: 0.9985 - val_loss: 0.0579 - val_accuracy: 0.9860
Epoch 

In [50]:
from tensorflow.keras import regularizers
#kernel_regularizer = regularizers.l2(0.001)

## Сохранение модели keras

In [51]:
model.save('keras_model.h5')

## Загрузка готовой модели keras

In [52]:
m1 = models.load_model('keras_model.h5')

In [53]:
text = X_long_box[2767]
print (text)
v = prepare_letter(text)
print (v)
vec = cv1.transform([v]).toarray()
vsc = SC.transform(vec)

Married local women looking for discreet action now! 5 real matches instantly to your phone. Text MATCH to 69969 Msg cost 150p 2 stop txt stop BCMSFWC1N3XX
marri local women look discreet action real match instantli phone text match msg cost p stop txt stop bcmsfwc n xx


In [54]:
m1.predict(vsc)

1/1 [==============================] - 1s 1s/step


array([[1.]], dtype=float32)